In [14]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from dask4in2p3.dask4in2p3 import Dask4in2p3

# Creating a Dask4in2p3 object  (with a default Python virtual environment)
#dask4in2p3 = Dask4in2p3()
dask4in2p3 = Dask4in2p3(virtual_env="/sps/lsst/users/boutigny/conda_envs/desc_dev")

import dask.dataframe as dd

In [15]:
# Launching a dask-scheduler and a dask-worker. 
# Connecting a client from this notebook to the dask-scheduler 
client = dask4in2p3.new_client(dask_worker_jobs=100, 
                               dask_worker_partition='dask',
                               dask_worker_memory=5,
                               dask_worker_time='00:30:00',
                               wait_for_workers_timeout=300
                              )

16:31:44,615 INFO    Creating and launching the SLURM jobs(s)
16:31:44,677 INFO    Waiting for the dask-scheduler SLURM job to be in RUNNING status
16:31:54,798 INFO    I've got the dask-scheduler SLURM job in RUNNING status
16:31:54,801 INFO    Waiting for the dask-worker SLURM job(s) to be in RUNNING status
16:33:25,310 INFO    I've got the expected count (=100) dask-worker SLURM job(s) in RUNNING status
16:33:25,379 INFO    A Dask client has been provided


In [16]:
client

<Client: 'tls://134.158.170.15:65485' processes=100 threads=100, memory=466.00 GiB>

In [17]:
# Read the full skysim5000 dataset (1572 parquet files)
#data_path = "/sps/lsst/groups/desc/skysim5000_v1.1.1_native/skysim5000_v1.1.1_healpix*.parquet"
# Read a subset of 251 skysim5000 parquet files
data_path = "/sps/lsst/groups/desc/skysim5000_v1.1.1_native/skysim5000_v1.1.1_healpix6*.parquet"

columns = ["ra", "dec", "halo_id", "halo_mass", "redshift", "mag_g", "mag_r", "mag_i", "mag_z", "mag_y", "is_central",
           "convergence", "baseDC2/sod_halo_mass", "baseDC2/is_on_red_sequence_gr", "baseDC2/is_on_red_sequence_ri"]

df = dd.read_parquet(data_path, columns=columns, engine='pyarrow', kwargs={'dataset': {'use_legacy_dataset': False}})

In [18]:
min_halo_mass = 10**13 #Msun

truth_data = df[df.halo_mass>min_halo_mass]
halo_data = df[(df.halo_mass>min_halo_mass) & (df.is_central==True)]
galaxy_data_all = df[(df.halo_mass>min_halo_mass) & (df.mag_i<30)]   

In [19]:
print("Number of elements in the truth catalog = ", len(galaxy_data_all))

Number of elements in the truth catalog =  18146630


In [20]:
print("Number of halos in the truth catalog = ", len(halo_data))

Number of halos in the truth catalog =  836159


In [21]:
halo_data.head()

,ra,dec,halo_id,halo_mass,redshift,mag_g,mag_r,mag_i,mag_z,mag_y,is_central,convergence,baseDC2/sod_halo_mass,baseDC2/is_on_red_sequence_gr,baseDC2/is_on_red_sequence_ri
177,1.819182,-0.450413,5506080464,1.290993e+13,0.058944,16.966107,16.095616,15.690103,15.424283,15.214057,True,-0.000080,8.044127e+12,True,True
229,2.383940,-0.220196,9406080464,1.937129e+13,0.067805,16.429710,15.567368,15.166042,14.902142,14.701941,True,-0.000152,1.047049e+13,True,True
1517,1.225478,-0.886729,11706080432,1.478354e+13,0.148846,18.006353,16.988222,16.517527,16.238794,16.016903,True,-0.000586,9.500311e+12,True,True
1602,0.495688,-0.409625,18806080432,1.142638e+13,0.134254,18.106377,17.196547,16.800372,16.542525,16.365241,True,0.000690,7.949881e+12,True,True
1662,2.389301,-0.155257,23106080432,1.586087e+13,0.134594,18.223822,17.372300,16.993462,16.721711,16.554072,True,0.000254,8.916359e+12,False,False


In [13]:
galaxy_data_all.head()

,ra,dec,halo_id,halo_mass,redshift,mag_g,mag_r,mag_i,mag_z,mag_y,is_central,convergence,baseDC2/sod_halo_mass,baseDC2/is_on_red_sequence_gr,baseDC2/is_on_red_sequence_ri
177,1.819182,-0.450413,5506080464,1.290993e+13,0.058944,16.966107,16.095616,15.690103,15.424283,15.214057,True,-0.000080,8.044127e+12,True,True
178,1.858688,-0.390812,5506080464,1.290993e+13,0.060835,21.785960,21.377345,21.124830,20.989822,20.903354,False,-0.000014,8.044127e+12,False,False
179,1.844373,-0.463940,5506080464,1.290993e+13,0.059392,17.831878,16.985828,16.540311,16.156551,15.960591,False,-0.000195,8.044127e+12,True,True
180,1.786733,-0.418667,5506080464,1.290993e+13,0.059590,20.417810,19.703201,19.257644,18.908508,18.715938,False,0.000243,8.044127e+12,False,False
181,1.814883,-0.470912,5506080464,1.290993e+13,0.060243,21.280379,20.570048,20.177962,19.859733,19.682847,False,-0.000030,8.044127e+12,False,False


In [24]:
galaxy_data_all.redshift

Dask Series Structure:
npartitions=251
    float64
        ...
     ...   
        ...
        ...
Name: redshift, dtype: float64
Dask Name: getitem, 2008 tasks

In [26]:
def mstar_i(redshift):
    bin = (redshift-0.020).round(2)/0.010
    bin = bin.round()+1
    bin = bin.mask(bin > 248, 248)
    return bin

def mstar_z(redshift):
    bin = redshift.round(2)/0.05
    bin = bin.round()
    bin = bin.mask(bin>55, 55)
    return bin

In [27]:
galaxy_data_all['index_i']=mstar_i(galaxy_data_all.redshift)
galaxy_data_all['index_z']=mstar_z(galaxy_data_all.redshift)

In [28]:
galaxy_data_all.head()

,ra,dec,halo_id,halo_mass,redshift,mag_g,mag_r,mag_i,mag_z,mag_y,is_central,convergence,baseDC2/sod_halo_mass,baseDC2/is_on_red_sequence_gr,baseDC2/is_on_red_sequence_ri,index_i,index_z
177,1.819182,-0.450413,5506080464,1.290993e+13,0.058944,16.966107,16.095616,15.690103,15.424283,15.214057,True,-0.000080,8.044127e+12,True,True,5.0,1.0
178,1.858688,-0.390812,5506080464,1.290993e+13,0.060835,21.785960,21.377345,21.124830,20.989822,20.903354,False,-0.000014,8.044127e+12,False,False,5.0,1.0
179,1.844373,-0.463940,5506080464,1.290993e+13,0.059392,17.831878,16.985828,16.540311,16.156551,15.960591,False,-0.000195,8.044127e+12,True,True,5.0,1.0
180,1.786733,-0.418667,5506080464,1.290993e+13,0.059590,20.417810,19.703201,19.257644,18.908508,18.715938,False,0.000243,8.044127e+12,False,False,5.0,1.0
181,1.814883,-0.470912,5506080464,1.290993e+13,0.060243,21.280379,20.570048,20.177962,19.859733,19.682847,False,-0.000030,8.044127e+12,False,False,5.0,1.0


In [32]:
richness = galaxy_data_all.groupby(['halo_id']).size().to_frame(name = 'richness').reset_index()

In [33]:
richness.head()

,halo_id,richness
0,106080279,9
1,106080411,7
2,306080315,12
3,406080224,11
4,806080347,79


In [42]:
halo_data.head()

,ra,dec,halo_id,halo_mass,redshift,mag_g,mag_r,mag_i,mag_z,mag_y,is_central,convergence,baseDC2/sod_halo_mass,baseDC2/is_on_red_sequence_gr,baseDC2/is_on_red_sequence_ri
177,1.819182,-0.450413,5506080464,1.290993e+13,0.058944,16.966107,16.095616,15.690103,15.424283,15.214057,True,-0.000080,8.044127e+12,True,True
229,2.383940,-0.220196,9406080464,1.937129e+13,0.067805,16.429710,15.567368,15.166042,14.902142,14.701941,True,-0.000152,1.047049e+13,True,True
1517,1.225478,-0.886729,11706080432,1.478354e+13,0.148846,18.006353,16.988222,16.517527,16.238794,16.016903,True,-0.000586,9.500311e+12,True,True
1602,0.495688,-0.409625,18806080432,1.142638e+13,0.134254,18.106377,17.196547,16.800372,16.542525,16.365241,True,0.000690,7.949881e+12,True,True
1662,2.389301,-0.155257,23106080432,1.586087e+13,0.134594,18.223822,17.372300,16.993462,16.721711,16.554072,True,0.000254,8.916359e+12,False,False


In [34]:
halo_data = dd.merge(halo_data, richness, on='halo_id')

In [35]:
%%time
res = halo_data.compute()
res

CPU times: user 635 ms, sys: 355 ms, total: 989 ms
Wall time: 41 s


,ra,dec,halo_id,halo_mass,redshift,mag_g,mag_r,mag_i,mag_z,mag_y,is_central,convergence,baseDC2/sod_halo_mass,baseDC2/is_on_red_sequence_gr,baseDC2/is_on_red_sequence_ri,richness
0,1.819182,-0.450413,5506080464,1.290993e+13,0.058944,16.966107,16.095616,15.690103,15.424283,15.214057,True,-0.000080,8.044127e+12,True,True,6
1,2.383940,-0.220196,9406080464,1.937129e+13,0.067805,16.429710,15.567368,15.166042,14.902142,14.701941,True,-0.000152,1.047049e+13,True,True,14
2,1.225478,-0.886729,11706080432,1.478354e+13,0.148846,18.006353,16.988222,16.517527,16.238794,16.016903,True,-0.000586,9.500311e+12,True,True,7
3,0.495688,-0.409625,18806080432,1.142638e+13,0.134254,18.106377,17.196547,16.800372,16.542525,16.365241,True,0.000690,7.949881e+12,True,True,11
4,2.389301,-0.155257,23106080432,1.586087e+13,0.134594,18.223822,17.372300,16.993462,16.721711,16.554072,True,0.000254,8.916359e+12,False,False,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,64.312525,-7.754377,4942706999121,1.061425e+13,3.026516,28.408617,26.922419,26.164019,25.774946,25.752455,True,0.031438,7.894443e+12,True,True,10
3616,65.005365,-7.920216,5103506999121,2.439882e+13,3.028372,30.436373,28.389220,27.111304,26.535443,26.499741,True,-0.008630,1.562627e+13,True,True,12
3617,65.548701,-8.184701,5184606999121,1.237647e+13,2.968906,25.102445,24.849855,24.721751,24.678056,24.750899,True,-0.036890,9.925340e+12,False,False,12
3618,63.815980,-8.282589,5547606999121,1.590771e+13,2.971272,28.535028,27.258888,26.372949,25.958667,25.899204,True,-0.040686,1.248105e+13,True,True,14


In [36]:
dask4in2p3.close()

2022-11-21 16:53:17,163 - distributed.client - ERROR - Failed to reconnect to scheduler after 90.00 seconds, closing client
